In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math as m
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('../input/ion-switching-shifting/train_clean_shifted.csv')
test=pd.read_csv('../input/ion-switching-shifting/test_clean_shifted.csv')
submission=pd.read_csv('../input/liverpool-ion-switching/sample_submission.csv')

train=train.sort_values(by=['signal'])
train=train.reset_index()

train_signal=train.signal.tolist()
train_signal.insert(0,float('-inf'))
train_signal.append(float('inf'))

train_channels=train.open_channels.tolist()
train_channels.insert(0,None)
train_channels.append(None)

k=2236
def Dist(arr1,arr2):
    total=0
    for i in range(len(arr1)):
        if(m.isnan(arr1[i]) or m.isnan(arr2[i])):
            continue
#         total+=min(1,abs(arr1[i]-arr2[i]))
        total+=abs(arr1[i]-arr2[i])
    return total

def binarySearch(arr,x): 
    l=0
    r=len(arr)-1
    
    while l <= r: 
  
        mid = l + (r - l)//2; 
          
        # Check if x is present at mid 
        if arr[mid] == x: 
            return (mid,mid+1)
  
        # If x is greater, ignore left half 
        elif arr[mid] < x: 
            l = mid + 1
  
        # If x is smaller, ignore right half 
        else: 
            r = mid - 1
      
    # If we reach here, then the element was not present 
    return (r,l)

Max=max(train['open_channels'])
Min=min(train['open_channels'])
# plt.scatter(train['time'],train['signal'],c=train['open_channels'].apply(lambda x:['darkred','red','orange','yellow','green','blue','indigo','violet','black','white','grey'][x]))

# plt.plot(test['time'],test['signal'],c='blue')
# plt.show()
for i in range(len(test)):
    Total=[0]*(Max-Min+1)
    count=0
    
    signal=test.at[i,'signal']
    result=binarySearch(train_signal,signal)
    
    L,R=result
    while count<=k:
        if(L<0):
            while count <=k:
                Total[train_channels[R]]+=1
                R+=1
                count+=1
            break
        
        if(R>len(test)-1):
            while count <=k:
                Total[train_channels[L]]+=1
                L-=1
                count+=1
            break
                
        if Dist([train_signal[L]],[signal])> Dist([train_signal[R]],[signal]):
            Total[train_channels[R]]+=1
            R+=1
            count+=1
        else: 
            Total[train_channels[L]]+=1
            L-=1
            count+=1

    pred=Total.index(max(Total))
#     pred=0
#     for i in range(len(Total)):
#         pred+=i*Total[i]
#     pred=round(pred/k)
    submission.at[i,'open_channels']=pred

print(submission)
submission.to_csv('Submission_Ion_Switching.csv',index=False,float_format='%.4f')